In [2]:
import numpy as np
import pandas as pd

In [3]:
item_cats_df = pd.read_csv('../data/input/item_categories.csv')
item_df = pd.read_csv('../data/input/items.csv')
sales_train_df = pd.read_csv('../data/input/sales_train.csv')
sample_submission_df = pd.read_csv('../data/input/sample_submission.csv')
shops_df = pd.read_csv('../data/input/shops.csv')

test = pd.read_csv('../data/input/test.csv')

In [45]:
print('item_categories')
print(item_cats_df.info())
display(item_cats_df.head())
print('***************************')
print('item')
print(item_df.info())
display(item_df.head())
print('***************************')
print('sales_train')
print(sales_train_df.info())
display(sales_train_df.head())
print('***************************')
print('shops')
print(shops_df.info())
display(shops_df.head())
print('***************************')
print('sample_submission')
print(sample_submission_df.info())
display(sample_submission_df.head())
print('***************************')
print('test')
print(test.info())
display(test.head())

item_categories
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
None


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


***************************
item
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB
None


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


***************************
sales_train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB
None


...

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


***************************
sample_submission
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB
None


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


***************************
test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB
None


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [54]:
np.sort(item_cats_df['item_category_id'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83],
      dtype=int64)

In [55]:
np.sort(item_df['item_category_id'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83],
      dtype=int64)

In [73]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [66]:
sales_train_df['date'] = pd.to_datetime(sales_train_df['date'])
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


date_block_num:連続した月の番号。　2013年1月=0, 2013年2月=1, ..., 2015年10月=33  
item_cnt_day:商品の売れた数。  
item_id:商品個別の識別子  
sales_train_df以外のデータは必要なし？

# item_cnt_dayの負の値をどう解釈するか

In [72]:
sales_train_df[sales_train_df['item_cnt_day'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2,2013-05-01,0,25,2552,899.0,-1.0
148,2013-01-23,0,25,2321,999.0,-1.0
175,2013-07-01,0,25,2199,1449.0,-1.0
807,2013-02-01,0,25,2330,599.0,-1.0
1041,2013-01-13,0,25,5034,1989.0,-1.0
...,...,...,...,...,...,...
2934243,2015-10-26,33,25,3917,449.0,-1.0
2934462,2015-10-18,33,25,4896,6398.0,-1.0
2935263,2015-05-10,33,25,10039,249.0,-1.0
2935643,2015-10-16,33,25,7893,2990.0,-1.0


In [87]:
print('item_cnt_day < 0:', len(sales_train_df[sales_train_df['item_cnt_day'] < 0]))
print('item_cnt_day:', len(sales_train_df))

item_cnt_day < 0: 7356
item_cnt_day: 2935849


In [105]:
tmp = sales_train_df[sales_train_df['item_cnt_day'] < 0]
print(np.sort(tmp['item_cnt_day'].unique()))

[-22. -16.  -9.  -6.  -5.  -4.  -3.  -2.  -1.]


In [128]:
tmp[tmp['item_cnt_day'] < -5]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
59156,2013-01-18,0,12,1590,2599.000000,-6.0
940092,2013-10-19,9,42,3732,2599.000000,-6.0
1057907,2013-02-11,10,12,8023,15.000000,-22.0
1835454,2014-10-07,18,14,20949,5.000000,-16.0
2048519,2014-10-10,21,12,9242,1433.333333,-9.0


In [122]:
from datetime import datetime
tmp_2 = sales_train_df[(sales_train_df['date'] < datetime(2013, 2, 11, 0, 0))\
               & (sales_train_df['item_id'] == 8023)]
print('2013-02-11以前のitem_id=8023の売上情報')
display(tmp_2)
print('item_cnt_day sum', tmp_2.item_cnt_day.sum())

2013-02-11以前のitem_id=8023の売上情報


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
11064,2013-02-01,0,19,8023,20.0,1.0
15486,2013-02-01,0,28,8023,20.0,1.0
29396,2013-01-13,0,27,8023,20.0,1.0
29397,2013-01-19,0,27,8023,20.0,2.0
29398,2013-01-21,0,27,8023,20.0,2.0
29399,2013-01-26,0,27,8023,20.0,1.0
57971,2013-02-01,0,14,8023,20.0,1.0
57973,2013-01-16,0,14,8023,20.0,3.0
220011,2013-01-02,1,28,8023,20.0,1.0
308578,2013-02-03,2,27,8023,20.0,1.0


item_cnt_day sum 28.0


In [131]:
tmp_3 = sales_train_df[(sales_train_df['date'] < datetime(2014, 10, 7, 0, 0))\
                      & (sales_train_df['item_id'] == 20949)]
print('2014-10-7以前のitem_id=20949の売上情報')
display(tmp_3)
print('item_cnt_day sum', tmp_3.item_cnt_day.sum())

2014-10-7以前のitem_id=20949の売上情報


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
352379,2013-04-30,3,22,20949,5.000000,12.0
352399,2013-04-29,3,22,20949,4.923077,13.0
352400,2013-04-28,3,22,20949,5.000000,4.0
352401,2013-04-27,3,22,20949,5.000000,4.0
352455,2013-04-24,3,22,20949,5.000000,2.0
...,...,...,...,...,...,...
2314861,2014-05-12,23,22,20949,5.000000,5.0
2314862,2014-04-12,23,22,20949,5.000000,9.0
2314863,2014-03-12,23,22,20949,5.000000,3.0
2314864,2014-02-12,23,22,20949,5.000000,3.0


item_cnt_day sum 124544.0
